In [2]:
import pandas as pd
df = pd.read_csv('datasets/clean_movies_dataset.csv')
df2 = pd.read_csv('datasets/credits.csv', index_col= 0)

In [3]:
df.columns

Index(['belongs_to_collection', 'budget', 'genres', 'id', 'original_language',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'release_year', 'return'],
      dtype='object')

### def cantidad_filmaciones_mes( Mes ): Se ingresa un mes en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en el mes consultado en la totalidad del dataset.
                    Ejemplo de retorno: X cantidad de películas fueron estrenadas en el mes de X

In [4]:
def cantidad_filmaciones_mes(mes: str):
    mes = mes.lower()
    df.release_date = pd.to_datetime(df.release_date)
    def mes_a_numero (mes):
        meses = {
            'enero':1,'febrero':2,'marzo':3,'abril':4,
            'mayo':5,'junio':6,'julio':7,'agosto':8,
            'septiembre':9,'octubre':10,'noviembre':11,'diciembre':12
            }
        return meses[mes]
    cantidad = df.release_date.loc[df.release_date.dt.month == mes_a_numero(mes)].count()
    return f'{cantidad.item()} cantidad de películas fueron estrenadas en el mes de {mes}'

- prueba

In [5]:
cantidad_filmaciones_mes('febrero')

'3032 cantidad de películas fueron estrenadas en el mes de febrero'

### def cantidad_filmaciones_dia( Dia ): Se ingresa un día en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en día consultado en la totalidad del dataset.
                    Ejemplo de retorno: X cantidad de películas fueron estrenadas en los días X

In [6]:
def cantidad_filmaciones_dia(dia : str):
    dia = dia.lower()
    df.release_date = pd.to_datetime(df.release_date, errors= 'coerce')
    def dia_numero(dia_numero):
        dias = {'lunes' : 0,'martes' : 1, 'miercoles': 2, 'jueves':3, 'viernes':4, 'sabado':5, 'domingo':6}
        return dias[dia_numero]
    cantidad = df.release_date.loc[df.release_date.dt.weekday == dia_numero(dia)].count()
    return {'dia' : dia , 'cantidad' : cantidad.item()}

- prueba

In [7]:
cantidad_filmaciones_dia('lunes')

{'dia': 'lunes', 'cantidad': 3503}

### def score_titulo( titulo_de_la_filmación ): Se ingresa el título de una filmación esperando como respuesta el título, el año de estreno y el score.
                    Ejemplo de retorno: La película X fue estrenada en el año X con un score/popularidad de

In [8]:
def score_titulo(titulo_de_la_filmacion: str):
    titulo_de_la_filmacion = titulo_de_la_filmacion.lower()
    respuesta = df[['title','release_year','popularity']].loc[df.title.apply(lambda x : x.lower()) == titulo_de_la_filmacion]
    titulo = respuesta['title'].values[0]
    anio = respuesta['release_year'].values[0]
    popularidad = respuesta['popularity'].values[0]
    return f"la pelicula {str(titulo)} fue estrenada en el año {str(anio)} con un score/popularidad de {str(popularidad)}"

- prueba

In [9]:
score_titulo('Toy Story')

'la pelicula Toy Story fue estrenada en el año 1995 con un score/popularidad de 21.946943'

### def votos_titulo( titulo_de_la_filmación ): Se ingresa el título de una filmación esperando como respuesta el título, la cantidad de votos y el valor promedio de las votaciones. La misma variable deberá de contar con al menos 2000 valoraciones, caso contrario, debemos contar con un mensaje avisando que no cumple esta condición y que por ende, no se devuelve ningun valor.
                    Ejemplo de retorno: La película X fue estrenada en el año X. La misma cuenta con un total de X valoraciones, con un promedio de X

In [10]:
def votos_titulo(titulo_de_la_filmacion: str):
    titulo_de_la_filmacion = titulo_de_la_filmacion.lower()
    respuesta = df[['title','release_year','vote_count','vote_average']].loc[df.title.apply(lambda x : x.lower()) == titulo_de_la_filmacion]
    titulo = respuesta['title'].values[0]
    anio = respuesta['release_year'].values[0]
    cantidad_votos = respuesta['vote_count'].values[0]
    promedio_votos = respuesta['vote_average'].values[0]
    if respuesta['vote_count'].values[0] <2000:
        return f'La variable ingresada no cumple con la condición necesaria de tener más de 2000 valoraciones'
    else:
        return f"la pelicula {str(titulo)} fue estrenada en el año {str(anio)}. La misma cuenta con {str(cantidad_votos)} votos con un promedio de {str(promedio_votos)}"


- prueba

In [11]:
votos_titulo('Toy Story')

'la pelicula Toy Story fue estrenada en el año 1995. La misma cuenta con 5415 votos con un promedio de 7.7'

### def get_actor( nombre_actor ): Se ingresa el nombre de un actor que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, la cantidad de películas que en las que ha participado y el promedio de retorno. La definición no deberá considerar directores.
                    Ejemplo de retorno: El actor X ha participado de X cantidad de filmaciones, el mismo ha conseguido un retorno de X con un promedio de X por filmación

In [12]:
def get_actor(nombre_actor: str):
    return f'El actor {x} ha participado de {x} cantidad de filmaciones, el mismo ha conseguido un retorno de {x} con un promedio de {x} por filmacion'

para este codigo necesito usar credits.csv, voy a revisarlo primero

In [13]:
df2.crew[1]

"[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'department': 'Production', 'gender': 2, 'id': 511, 'job': 'Executive Producer', 'name': 'Larry J. Franco', 'profile_path': None}, {'credit_id': '52fe44bfc3a36847f80a7c89', 'department': 'Writing', 'gender': 2, 'id': 876, 'job': 'Screenplay', 'name': 'Jonathan Hensleigh', 'profile_path': '/l1c4UFD3g0HVWj5f0CxXAvMAGiT.jpg'}, {'credit_id': '52fe44bfc3a36847f80a7cdd', 'department': 'Sound', 'gender': 2, 'id': 1729, 'job': 'Original Music Composer', 'name': 'James Horner', 'profile_path': '/oLOtXxXsYk8X4qq0ud4xVypXudi.jpg'}, {'credit_id': '52fe44bfc3a36847f80a7c7d', 'department': 'Directing', 'gender': 2, 'id': 4945, 'job': 'Director', 'name': 'Joe Johnston', 'profile_path': '/fok4jaO62v5IP6hkpaaAcXuw2H.jpg'}, {'credit_id': '52fe44bfc3a36847f80a7cd7', 'department': 'Editing', 'gender': 2, 'id': 4951, 'job': 'Editor', 'name': 'Robert Dalva', 'profile_path': None}, {'credit_id': '573523bec3a368025100062c', 'department': 'Production', 'gender': 0, '

- prueba

In [14]:
get_actor()

TypeError: get_actor() missing 1 required positional argument: 'nombre_actor'

### def get_director( nombre_director ): Se ingresa el nombre de un director que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, deberá devolver el nombre de cada película con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma.

In [ ]:
def get_director(nombre_director: str):
    return None

- prueba

In [ ]:
get_director()